In [2]:
from bs4 import BeautifulSoup
import requests
import time

#  Part 1 Scraping and Saving HTML Content

## 1. Identify the Target


In [54]:
free_url = 'https://sfbay.craigslist.org/search/zip#search=1~gallery~0~0'
headers = {'User-Agent': 'Mozilla/5.0'}
free_page = requests.get(free_url)
free_soup = BeautifulSoup(free_page.text, 'html.parser')
print(free_soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width,initial-scale=1" name="viewport"/>
  <meta content="craigslist" property="og:site_name"/>
  <meta content="preview" name="twitter:card"/>
  <meta content="SF bay area free stuff - craigslist" property="og:title"/>
  <meta content="SF bay area free stuff - craigslist" name="description"/>
  <meta content="SF bay area free stuff - craigslist" property="og:description"/>
  <meta content="https://sfbay.craigslist.org/search/zip" property="og:url"/>
  <title>
   SF bay area free stuff - craigslist
  </title>
  <link href="https://sfbay.craigslist.org/search/zip" rel="canonical"/>
  <link href="https://sfbay.craigslist.org/search/zip" hreflang="x-default" rel="alternate"/>
  <link href="/favicon.ico" id="favicon" rel="icon">
   <script id="ld_searchpage_data" type="application/ld+json">
    {"description":"Free Stuff in SF Bay Area","@context":

## 2. Interact with the Page-Sorting:

In [4]:
time.sleep(5) # 5s
oldest_url = 'https://sfbay.craigslist.org/search/zip?sort=dateoldest#search=1~gallery~0~0'
pldest_page = requests.get(oldest_url)

This is a get request, which includes using an URL to request information from the server. The variable associated with sorting is "sort". Value of sort will be "dateoldest" once change to sort by oldest. 

# 3. Interact with the Page-Pagination:

## Explaination:
On the first page, the URL has #search = 1-gallery-0-0, and then on the second page, it changes to 1-gallery-1-0, then 1-gallery-2-0 for third page. 

# 4. Fetch Listing URLs:

In [55]:
time.sleep(5) # 5s
newest_url = 'https://sfbay.craigslist.org/search/zip?sort=date#search=1~gallery~0~0'
newest_page = requests.get(newest_url)
newest_soup = BeautifulSoup(newest_page.text, 'html.parser')
# print(newest_soup)

allListings = []
# find all "li"
listings= newest_soup.find_all('li', class_ = 'cl-static-search-result')
for li in listings:
    # in each "li", find the link 
    tag = li.find('a')
    allListings.append(tag['href'])
print(allListings)

['https://sfbay.craigslist.org/eby/zip/d/livermore-white-square-coffee-table/7715696695.html', 'https://sfbay.craigslist.org/eby/zip/d/richmond-glass-bottles/7715695864.html', 'https://sfbay.craigslist.org/sfc/zip/d/san-francisco-child-guard-plastic-bag/7712999768.html', 'https://sfbay.craigslist.org/eby/zip/d/lafayette-free-kids-bike/7715693972.html', 'https://sfbay.craigslist.org/nby/zip/d/sausalito-free-cb2-desk-cb2-lamp-pick/7715693686.html', 'https://sfbay.craigslist.org/eby/zip/d/livermore-ethan-allen-eight-drawer/7715691591.html', 'https://sfbay.craigslist.org/eby/zip/d/oakland-free-book-stand/7715691225.html', 'https://sfbay.craigslist.org/sfc/zip/d/san-francisco-very-mediocre-pumpkin-pie/7714736797.html', 'https://sfbay.craigslist.org/sfc/zip/d/san-francisco-kitty-sill-cat-window/7715689015.html', 'https://sfbay.craigslist.org/sfc/zip/d/san-francisco-stereo-home-entertainment/7715688899.html', 'https://sfbay.craigslist.org/eby/zip/d/san-leandro-free-wood-bifold-closet/77156876

I found that each listing link is included in a "li" which has the same class name "cl-static-search-result". So I found each li that has this class name, and inside of each "li", I extract the url and put it in a list "allListings".

# one more possible selection

This finds all 'a' links that are direct children of each 'li' objects which are in cl-static-search-result class. 

In [32]:
links = newest_soup.select('li.cl-static-search-result > a')
for tag in links:
    allListings.append(tag['href'])
print(allListings)

['https://sfbay.craigslist.org/eby/zip/d/oakland-free-arms-reach-co-sleeper-and/7715210741.html', 'https://sfbay.craigslist.org/sby/zip/d/cupertino-free-shoes/7715210537.html', 'https://sfbay.craigslist.org/sby/zip/d/los-gatos-washer-and-dryer/7715210460.html', 'https://sfbay.craigslist.org/sby/zip/d/los-gatos-free-desk/7715210115.html', 'https://sfbay.craigslist.org/sby/zip/d/los-gatos-free-wood-an-door/7715209887.html', 'https://sfbay.craigslist.org/pen/zip/d/stanford-chair/7715209498.html', 'https://sfbay.craigslist.org/eby/zip/d/richmond-4x8-cutting-matt/7715209378.html', 'https://sfbay.craigslist.org/nby/zip/d/san-anselmo-cat-haven/7715208169.html', 'https://sfbay.craigslist.org/eby/zip/d/berkeley-large-all-wood-desk/7715207593.html', 'https://sfbay.craigslist.org/eby/zip/d/concord-three-free-wine-cabinets/7715205223.html', 'https://sfbay.craigslist.org/eby/zip/d/berkeley-rustic-wheelbarrow/7715205056.html', 'https://sfbay.craigslist.org/nby/zip/d/novato-double-wide-filing-cabinet

# first 250 lists

In [64]:
max_listings = 250
page_number = 0
items_per_page = 120
allListings = []

# only loop until before length of all listings reach at 250
while len(allListings) < max_listings:
    #s parameter determines the starting index
    url = f'https://sfbay.craigslist.org/search/zip?s={page_number * items_per_page}&sort=date#search=1~gallery~{page_number}~0'
    time.sleep(5) # 5s
    free_page = requests.get(url)
    free_soup = BeautifulSoup(free_page.text, 'html.parser')
    # all "li"
    listings = free_soup.find_all('li', class_='cl-static-search-result')
    
    for li in listings:
        tag = li.find('a')
        allListings.append(tag['href'])
        if len(allListings) == max_listings: # after reached 250, the loop breaks
                break
    # after all the listings get fecthed to "alllistings" on one page, page number jumps to the next
    page_number = page_number + 1

print(len(allListings))
print(allListings)

250
['https://sfbay.craigslist.org/nby/zip/d/santa-rosa-puppys/7715713871.html', 'https://sfbay.craigslist.org/sby/zip/d/san-jose-tundra-hub-caps-set-of-fits/7713712529.html', 'https://sfbay.craigslist.org/eby/zip/d/oakland-free-amana-electric-stove/7715713459.html', 'https://sfbay.craigslist.org/sby/zip/d/san-jose-adjustable-desk-chair/7713714866.html', 'https://sfbay.craigslist.org/nby/zip/d/windsor-estate-wine-opener/7715712237.html', 'https://sfbay.craigslist.org/scz/zip/d/felton-cal-king-size-bed-felton/7715711748.html', 'https://sfbay.craigslist.org/nby/zip/d/santa-rosa-shih-tzu-yrs/7715711501.html', 'https://sfbay.craigslist.org/nby/zip/d/san-anselmo-free-corner-desk/7715711331.html', 'https://sfbay.craigslist.org/scz/zip/d/freedom-moving-boxes-and-bubble-wrap/7715711311.html', 'https://sfbay.craigslist.org/pen/zip/d/daly-city-free-vintage-chair-and-end/7715711300.html', 'https://sfbay.craigslist.org/sby/zip/d/san-jose-chair-and-ottoman/7715709308.html', 'https://sfbay.craigslis

when page number sets to 0, which means the first page. And to get the first 250 lists, I use the while loop to run until the length of allListings is less than 250. the parameter "s" in the URL means that starting index on the current page. So for example on the second page, the starting index would be 1*120 = 120. Within each url, I do the same step as utilized earlier to extract the listing links in a for loop. and once the length of allListings reaches to 250, the loop breaks. 

# 5. Save HTML Pages:
For each of the 250 listing URLs, use `requests` to fetch the listing page.

Save each HTML content to a separate file on disk.  Use each listing’s ID to organize files in a way that makes them easily identifiable.

In [7]:
import requests
import os
from urllib.parse import urlparse, parse_qs

# create a new directory
directory_name = "craigslist_listings"
# get the current directory of this jupyter book
current_working_directory = os.getcwd()

# full path where the HTML files are
path = os.path.join(current_working_directory, directory_name)

# avoid error if the directory already exists
os.makedirs(path, exist_ok=True)

for url in allListings:
    time.sleep(5) # 5s
    url_response = requests.get(url)
    # parsing the url, break down the url into components
    parsed_url = urlparse(url)
    # basename is the last part (listing ID)
    listing_id = os.path.basename(parsed_url.path)
    # adding listing_id extensions after the path
    html_path = os.path.join(path, f"{listing_id}.html")
        
        # write the content of "url_response" to the html file
    with open(html_path, 'w', encoding='utf-8') as file:
        file.write(url_response.text)

# Part 2: Parsing and Displaying Information from Saved HTML

1. Write a script that reads each of the saved HTML files from the disk. 
2. Extract Information:
For each HTML file, use `BeautifulSoup` to parse the file content.

Extract and print the following details:

Title: The title of the listing.

URL of first image (if an image exists):  The URL of the displayed image.  It can be found in the `src` attribute of `<img>`

Description: The full description text of the listing.

Post ID: Usually found at the bottom of the page or within the page's HTML structure.

Posted Date: The date when the listing was originally posted.

Last Updated Date: The date when the listing was last updated.

In [65]:
# Loop through each file in the directory
number_file = 0
for filename in os.listdir(path):

    # Check if the file ends with .html

    if filename.endswith(".html"):

        # Construct the full file path

        filepath = os.path.join(path, filename)

        # Read file to string
        
        with open(filepath, 'r', encoding='utf-8') as file:
            
            html = file.read()
            soup = BeautifulSoup(html, 'html.parser')
            
            # title
            title_span = soup.find('span', id='titletextonly')
            if title_span:
                title = title_span.get_text(strip=True)
                next_span = title_span.find_next('span')
                if next_span:
                    title = title + next_span.get_text(strip=True)
            
            # URL for first image
            first_img = soup.find('img')
            if first_img and 'src' in first_img.attrs:
                img_src = first_img['src']
            
            # full discription
            desc_section = soup.find('section', id = 'postingbody')
            if desc_section:
                description = desc_section.get_text(strip=True)
                
            # posting ID
            div_postID = soup.find('div', class_ = 'postinginfos')
            if div_postID: 
                post_id = div_postID.find('p', class_='postinginfo').text
            
            # post Date
            post_date = ""
            div_post = soup.find('div', class_ = 'postinginfos')
            if div_post:
                posted_info = div_post.find_all('p', class_='postinginfo reveal')
                for info in posted_info:
                    # check if there is posted date
                    if 'posted' in info.text:
                        post_date = info.find('time', class_='date timeago').text.strip()
            
            # last updated
            last_updated = ""
            if div_post:
                updated_info = div_post.find_all('p', class_='postinginfo reveal')
                for info in updated_info:
                    # check if there is updated date
                    if 'updated' in info.text:
                        last_updated = info.find('time', class_='date timeago').text.strip()
                        break
            number_file =  number_file + 1   
            print(number_file)
            print("Title:", title)
            print("URL for First Image:", img_src)
            print("Full Description:", description)
            print(post_id)
            print("Posting Date:", post_date)
            print("Last Updated Date:", last_updated)
            print("-" * 50)
            
                        

1
Title: desk tops(san jose east)
URL for First Image: https://images.craigslist.org/00J0J_lrmbAgKZcus_0CI0t2_600x450.jpg
Full Description: QR Code Link to This Postcorner desk tops for projects salvageSome water damageor cubicle desk top no basevery large pieces they can be taken apart about 7 ft by 7ft topheavy duty still usable  I have 3 pieces leftoffice deskcubicleoffice chairlaminate top
post id: 7705755506
Posting Date: 2024-01-09 08:07
Last Updated Date: 2024-02-06 21:35
--------------------------------------------------
2
Title: Free scrap steel(richmond / point / annex)
URL for First Image: https://images.craigslist.org/00i0i_cjHJk0MQyLi_0CI0t2_600x450.jpg
Full Description: QR Code Link to This PostMaybe 300lbsCall/textFirst come first serve
post id: 7705932831
Posting Date: 2024-01-09 15:03
Last Updated Date: 2024-02-06 14:56
--------------------------------------------------
3
Title: Twin headboard(rohnert pk / cotati)
URL for First Image: https://images.craigslist.org/00C0

22
Title: Free firewood(san jose west)
URL for First Image: https://images.craigslist.org/00202_lMsHrd2HJZG_0t20CI_600x450.jpg
Full Description: QR Code Link to This PostFree redwood firewood
post id: 7712177478
Posting Date: 2024-01-28 15:17
Last Updated Date: 2024-02-06 12:26
--------------------------------------------------
23
Title: Wicker basket(east palo alto)
URL for First Image: https://images.craigslist.org/00M0M_iLQiupON4NP_0t20CI_600x450.jpg
Full Description: QR Code Link to This PostWicker tray/shallow basket. About 13.5” across.
post id: 7712183676
Posting Date: 2024-01-28 15:43
Last Updated Date: 2024-02-06 20:59
--------------------------------------------------
24
Title: Free Outdoor Patio Wood Burning Oven Fireplace Heater(santa cruz)
URL for First Image: https://images.craigslist.org/00Q0Q_gt284t8nPbV_0t20CI_600x450.jpg
Full Description: QR Code Link to This PostGreat working outdoor fireplace.  No more room for it.  It's heavy.  You move it and it's yours.  Fire too

43
Title: Free Mr brown bottles, nipple, fruit pacifier, etc.(novato)
URL for First Image: https://images.craigslist.org/00g0g_14e9kFCGwpt_1320MM_600x450.jpg
Full Description: QR Code Link to This PostFree Mr brown bottles, nipples, etc. The gas relif drop is brand new and the packahe is unopened. There is one pack of nipple#1 which is brand new and never used, the fruit pacifier was used only one time and the rest of the nipples never used. If you need them please let me know.
post id: 7714401335
Posting Date: 2024-02-04 11:32
Last Updated Date: 2024-02-06 14:38
--------------------------------------------------
44
Title: Genuine MWF Refrigerator Water Filter for GE(dublin / pleasanton / livermore)
URL for First Image: https://images.craigslist.org/00z0z_gIYT6DX1N8y_0oo0ww_600x450.jpg
Full Description: QR Code Link to This PostSee attached picture, brand new.  Pickup in Dublin.
post id: 7714480447
Posting Date: 2024-02-04 15:30
Last Updated Date: 2024-02-06 19:23
---------------------

67
Title: Hanging Lamp With Three Light Bulbs(santa rosa)
URL for First Image: https://images.craigslist.org/00m0m_2WfiUyO1mLO_0lM0t2_600x450.jpg
Full Description: QR Code Link to This PostHanging Lamp With Three Light Bulbs.  Needs shades (they broke) and a chain otherwise it works fine.
post id: 7715107304
Posting Date: 2024-02-06 12:42
Last Updated Date: 
--------------------------------------------------
68
Title: Hanging Lamp With Three Light Bulbs(santa rosa)
URL for First Image: https://images.craigslist.org/00m0m_2WfiUyO1mLO_0lM0t2_600x450.jpg
Full Description: QR Code Link to This PostHanging Lamp With Three Light Bulbs.  Needs shades (they broke) and a chain otherwise it works fine.
post id: 7715107304
Posting Date: 
Last Updated Date: 
--------------------------------------------------
69
Title: Finger puppets - FREE(mission district)
URL for First Image: https://images.craigslist.org/01515_pQSZq8sqVj_0CI0t2_600x450.jpg
Full Description: QR Code Link to This PostSmall collec

91
Title: Free Razer PS5 Headset‹
URL for First Image: https://images.craigslist.org/00j0j_5fz1sX61NHH_0ak0bl_600x450.jpg
Full Description: QR Code Link to This PostFree PS5 headset for anyone in need. Don’t need it anymore as I don’t play much on the PS5Call atshow contact info
post id: 7715140719
Posting Date: 2024-02-06 14:08
Last Updated Date: 
--------------------------------------------------
92
Title: Armchair / Recliner, Queen Anne Style Wing Chair, Good Condition(castro / upper market)
URL for First Image: https://images.craigslist.org/00D0D_kXYScm5tI0k_0t20CI_600x450.jpg
Full Description: QR Code Link to This PostArmchair, Queen Anne style wing chair that also reclines to two positions (looks like a regular armchair when not reclined).  Burgundy jacquard upholstery (see photos) with no rips, stains, etc. (always used arm covers).  All recliner mechanism works.  In Good-to-Excellent condition.  From a smoke- and pet-free home.  We're downsizing, otherwise we'd keep it.Nice cha

...

127
Title: anyone know a baby CLYDE? free monogrammed stuff(oakland west)
URL for First Image: https://images.craigslist.org/00h0h_hKExpriEG1a_07K0ak_600x450.jpg
Full Description: QR Code Link to This PostOur baby Clyde is now grown, so it is time to find a new baby Clyde who might want a rocking chair and little train set with his name on it. We also have a big boy shirt with Clyde monogrammed on the sleeve cuff. Would love to pass it on to another little Clyde!
post id: 7715165739
Posting Date: 2024-02-06 15:24
Last Updated Date: 
--------------------------------------------------
128
Title: Ping Driver(petaluma)
URL for First Image: https://images.craigslist.org/00K0K_gayrPrA6nBt_0t20CI_600x450.jpg
Full Description: QR Code Link to This PostGolf - Ping G-15 Driver, 10.5 degrees, regular flex
post id: 7715166576
Posting Date: 2024-02-06 15:27
Last Updated Date: 
--------------------------------------------------
129
Title: hill’s prescription diet kidney care cat kibble (unsealed)(wa

153
Title: Doggy supplements for joint and hip(richmond / seacliff)
URL for First Image: https://images.craigslist.org/00B0B_iodqAOWeAMe_0t20CI_600x450.jpg
Full Description: QR Code Link to This PostSpringtime brand "Advances Hip & Joint" supplements: Glucosamine, Chondroitin, MSM and Boswellia serrata in a beef liver pill. Three bottles, 60 tablets/bottle.
post id: 7715176021
Posting Date: 2024-02-06 16:01
Last Updated Date: 
--------------------------------------------------
154
Title: Child Safety Locks Brand New(hayward / castro valley)
URL for First Image: https://images.craigslist.org/00707_e4KpkGSboUe_0t20CI_600x450.jpg
Full Description: QR Code Link to This PostI bought these and never ended up using them. One of the packages has been opened but was never used. Total of 20 child safety locks for kitchen and bathroom doors and drawers.
post id: 7715177333
Posting Date: 2024-02-06 16:05
Last Updated Date: 
--------------------------------------------------
155
Title: Rockwell mot

177
Title: House & Garden Stuff(San Jose)
URL for First Image: https://images.craigslist.org/00B0B_fNQcUS0rme8_0t20CI_600x450.jpg
Full Description: QR Code Link to This PostAt curb.1560 Grace Avenue
post id: 7715190659
Posting Date: 2024-02-06 16:54
Last Updated Date: 
--------------------------------------------------
178
Title: Glass Door Cabinet(richmond / point / annex)
URL for First Image: https://images.craigslist.org/00e0e_64UogZ0JVVj_0CI0t2_600x450.jpg
Full Description: QR Code Link to This PostBetween McLaughlin and Lassen Streets on McBryde Avenue - I have not measured it - so photos show all I know - I don't carry a tape measure when walking my dog - I know the owner, but have not asked him about it - I just know that he is a craftsman with granite counter tops and may have gotten this from one of his work sites - it looks like it could be brand new - the back of the cabinet is a mirror and on the back appears to be manufacturer's part number and style numbers - it is wrappe

218
Title: Moving out , bed tiny desk , chair(inner richmond)
URL for First Image: https://images.craigslist.org/00z0z_hN4KpnihZ11_0CI0t2_600x450.jpg
Full Description: QR Code Link to This PostHello,I’m moving into a furnished room and am trying to get rid of my bed. Twin size , frame , everything is included (+a wrench to assemble).Extremely easy to put together. Adding a foam cushion that I used a few times (too soft for me), bed elevators for storage space . And will throw in a back rest.I don’t have a car so you’d have to pick up.Also have a tiny foldable desk and tall mirror if interested.Ty
post id: 7715226780
Posting Date: 2024-02-06 19:56
Last Updated Date: 
--------------------------------------------------
219
Title: Firewood(Mountain View)
URL for First Image: https://images.craigslist.org/00E0E_jAiwsmdNlZt_0t20CI_600x450.jpg
Full Description: QR Code Link to This PostLeftover firewood. Mostly hardwood. A bit damp from rains last week but should be nice once it dries out a b

239
Title: FREE: Small Brown Padded Bench(novato)
URL for First Image: https://images.craigslist.org/00k0k_9PnLEPI4rcf_05a03S_600x450.jpg
Full Description: QR Code Link to This Post23½"Long   x  13½" Width x  19"HeightEasy pick-up under carport
post id: 7715237910
Posting Date: 2024-02-06 21:28
Last Updated Date: 
--------------------------------------------------
240
Title: Free plant pots, crayons, Dominoes and various other items(excelsior / outer mission)
URL for First Image: https://images.craigslist.org/00808_8JmVQwJ2nW4_0CI0t2_600x450.jpg
Full Description: QR Code Link to This PostFree items for those who want them. All cleaned and seldom used.Let me know what you would like and I can leave outside for easy pick up.Let me know if you have questions.Thank you!
post id: 7715238476
Posting Date: 2024-02-06 21:33
Last Updated Date: 
--------------------------------------------------
241
Title: Empty bottles Pliny the Younger(twin peaks / diamond hts)
URL for First Image: https://ima

# Part 3: Automating Login on The Old Reader

# 1. Creating and Verifying a The Old Reader Account


# 2. Exploring the Login Mechanism

In [60]:
headers = {'User-agent': 'Mozilla/5.0'}
time.sleep(5) # 5s
reader_url = 'https://theoldreader.com/users/sign_in'
signin_page = requests.get(reader_url, headers=headers)
signin_soup = BeautifulSoup(signin_page.content, 'html.parser')
# print(signin_soup.prettify())

# <form accept-charset="UTF-8" action="/users/sign_in" class="new_user" id="new_user" method="post" role="form">
#          <div style="margin:0;padding:0;display:inline">
#           <input name="utf8" type="hidden" value="✓"/>
#           <input name="authenticity_token" type="hidden" value="eKAwUONGTR/hqkziqjeXVPD5U1JciM9wdIRooyxZsmY="/>
#          </div>
#          <div class="form-group">
#           <input autocapitalize="off" autocorrect="off" autofocus="autofocus" class="form-control" id="user_login" name="user[login]" placeholder="Username/Email" size="30" spellcheck="false" type="text"/>
#          </div>
#          <div class="form-group">
#           <input class="form-control" id="user_password" name="user[password]" placeholder="Password" size="30" type="password"/>
#          </div>
#          <input class="btn btn-primary btn-block" name="commit" type="submit" value="Sign In"/>
#         </form>

# 3. Analyzing Network Traffic for Login Request


Using username and password to submit and sign in and get information from the server in the body of the request is a POST request. And server returns with cookies.

# 4. Automating the Login Process
Using Python and appropriate libraries like requests, simulate the login process.

Create a session object to maintain your login state across multiple requests.

Prepare a payload with your login credentials and other necessary form data identified from the login page and the network analysis.

Send a POST request to the login form’s action URL to log in, using the session object.



In [61]:
time.sleep(5) # 5s
input = signin_soup.select_one('#new_user input[name=authenticity_token]')
_ref_ck = input.get('value')
print(_ref_ck)

RZtzYBL29gdmSIZXM7ftYn+lNMafXZU0dnDfQ1n0i14=


In [62]:
# Always pause between two requests.
time.sleep(5) # 5s

# An open session carries the cookies and allows you to make post requests
session_requests = requests.session()

res = session_requests.post('https://theoldreader.com/users/sign_in', 
                        data = {
                                'authenticity_token' : _ref_ck,
                                'user[login]' : '',
                                'user[password]' : ''}, 
                        timeout = 20)

cookies = session_requests.cookies.get_dict()
print(cookies)

{'remember_user_token': 'BAhbB1sGVToaTW9wZWQ6OkJTT046Ok9iamVjdElkIhG%2BKPE1yDkztuTzI%2BdJIiIkMmEkMDUkRS9iLlBKc0thZUpCeEouRGJrNUZMLgY6BkVU--9dbdf4cb8ff5f4c14631794d389a02b62eb07a33', 'i_know_you': 'prettycola', '_new_reader_session': 'BAh7CkkiD3Nlc3Npb25faWQGOgZFVEkiJTNkNjhjNDhmODk3MmViNThlNGY0YTIyZDQ0OThmNWE4BjsAVEkiGXdhcmRlbi51c2VyLnVzZXIua2V5BjsAVFsHWwZVOhpNb3BlZDo6QlNPTjo6T2JqZWN0SWQiEb4o8TXIOTO25PMj50kiIiQyYSQwNSRFL2IuUEpzS2FlSkJ4Si5EYms1RkwuBjsAVEkiDWxhbmd1YWdlBjsARjoHZW5JIhByZWRpcmVjdF90bwY7AEZJIgYvBjsARkkiEF9jc3JmX3Rva2VuBjsARkkiMXYxWjFmMmY0NGtJUmFOK2FwUjBCdkRSdmdNdWVEMkdhSk5RdkU1eWp3aGM9BjsARg%3D%3D--dc1b80e84c4a32750f2e9bbd4c48500b337c3aa6', 'signed_at': '1707414792'}


# 5. Verifying Successful Login
After attempting to log in, inspect the cookies saved in the session object to understand the information The Old Reader stores on your computer.

Use the session object to access https://theoldreader.comLinks to an external site..

Verify successful login by checking for the presence of your user information that is only available when logged in.

In [67]:
import re
time.sleep(5) # 5s

page2 = session_requests.get(reader_url, cookies=cookies)
soup2 = BeautifulSoup(page2.content, 'html.parser')
print(soup2.prettify())


<!DOCTYPE html>
<html>
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <!-- The above 3 meta tags *must* come first in the head; any other head content must come *after* these tags -->
  <link href="//s.theoldreader.com/assets/application-befb06d5a14978388154b51422cef437.css" media="all" rel="stylesheet" type="text/css"/>
  <link href="//s.theoldreader.com/assets/apple-touch-icon-57x57-86fe1176e14af4907a6fecfe5ca7e3f1.png" rel="apple-touch-icon-precomposed" sizes="57x57"/>
  <link href="//s.theoldreader.com/assets/apple-touch-icon-114x114-bae89acc41c93261dd962ea6ade08d22.png" rel="apple-touch-icon-precomposed" sizes="114x114"/>
  <link href="//s.theoldreader.com/assets/apple-touch-icon-72x72-f248503edfa3676f8d58af531aff7e88.png" rel="apple-touch-icon-precomposed" sizes="72x72"/>
  <link href="//s.theoldreader.com/assets/apple-touch-icon-144x144-510415291cae9b46a9ca4ac398